### ตัววางแผนแบบลำดับบางส่วน (Partial-Order Planner)
อัลกอริทึมการวางแผนแบบลำดับบางส่วนแตกต่างจากตัววางแผนแบบลำดับทั้งหมดอย่างมีนัยสำคัญ
วิธีทำงานของแผนลำดับบางส่วนช่วยให้ใช้ประโยชน์จากการแบ่งปัญหา (_problem decomposition_) และทำงานกับปัญหาย่อยแต่ละส่วนแยกกันได้
มันจะทำงานกับเป้าหมายย่อยหลายตัวอย่างเป็นอิสระ แก้ด้วยชุดแผนย่อยหลายชุด แล้วจึงรวมเป็นแผนเดียว
<br>
ตัววางแผนแบบลำดับบางส่วนยังยึดตามกลยุทธ์ **least commitment** โดยเลื่อนการตัดสินใจให้นานที่สุดเท่าที่จะทำได้
ตัวแปรจะไม่ถูกผูกค่าเว้นแต่จำเป็นอย่างยิ่ง และจะเลือกการกระทำใหม่ก็ต่อเมื่อการกระทำที่มีอยู่ไม่สามารถทำให้เงื่อนไขก่อนที่ต้องการเป็นจริงได้
<br>
อัลกอริทึมการวางแผนใดๆ ที่สามารถวางการกระทำสองอย่างลงในแผนโดยไม่ระบุว่าอันไหนมาก่อน เรียกว่า **ตัววางแผนแบบลำดับบางส่วน (partial-order planner)**
ตัววางแผนแบบลำดับบางส่วนทำการค้นหาในพื้นที่ของ “แผน” มากกว่าพื้นที่ของ “สถานะ” ซึ่งทำให้ทำงานได้ดีกว่าสำหรับปัญหาบางประเภท
<br>
<br>
ต่อไปมาดูคลาส `PartialOrderPlanner` กัน

In [3]:
from planning import *
from notebook import psource

In [4]:
psource(PartialOrderPlanner)

เราจะเริ่มจากอธิบายโครงสร้างข้อมูลและเมธอดผู้ช่วยที่ใช้ จากนั้นจึงอธิบายอัลกอริทึมที่ใช้ค้นหาแผนแบบลำดับบางส่วน

แผนแต่ละชุดประกอบด้วย 4 องค์ประกอบดังนี้:

1. **`actions`**: เซตของการกระทำที่เป็นขั้นตอนของแผน
`actions` มักเป็นสับเซตของ `pddl.actions` ซึ่งเป็นเซตของการกระทำที่เป็นไปได้สำหรับปัญหาการวางแผนที่กำหนด
การกระทำ `start` และ `finish` เป็นดัมมีแอ็กชันเพื่อทำให้ปัญหามีรูปแบบสม่ำเสมอ การกระทำ `start` ไม่มีเงื่อนไขก่อน และผลของมันคือสถานะเริ่มต้นของปัญหาการวางแผน
การกระทำ `finish` ไม่มีผล และเงื่อนไขก่อนของมันคือสถานะเป้าหมายของปัญหาการวางแผน
แผนว่าง (empty plan) ประกอบด้วยเพียงสองดัมมีแอ็กชันนี้
2. **`constraints`**: เซตของข้อจำกัดเชิงเวลา (temporal constraints) ที่กำหนดลำดับสัมพัทธ์ระหว่างการกระทำ
`constraints` ไม่ได้กำหนดลำดับเชิงเส้นโดยตรง แต่มักแทนด้วยกราฟมีทิศทาง ซึ่งถ้าแผนสอดคล้อง (consistent) กราฟจะต้องไม่มีวัฏจักร (acyclic)
แต่ละข้อจำกัดมีรูป A < B ซึ่งอ่านว่า “A มาก่อน B” และหมายถึงการกระทำ A _ต้อง_ ถูกทำก่อนการกระทำ B ในบางเวลา แต่ไม่จำเป็นต้องก่อนทันที
`constraints` เก็บข้อมูลนี้เป็นเซตของทูเพิล `(Action(A), Action(B))` ตามความหมายข้างต้น
ไม่สามารถเพิ่มข้อจำกัดใหม่ลงใน `constraints` ได้หากทำให้กราฟที่มีอยู่ไม่เป็น acyclic อีกต่อไป
3. **`causal_links`**: เซตของ causal-link
Causal link ระหว่างการกระทำ _A_ และ _B_ ในแผนเขียนเป็น _A_ --_p_--> _B_ อ่านว่า “A ทำให้ p สำเร็จสำหรับ B”
สิ่งนี้หมายความว่า _p_ เป็นเอฟเฟกต์ของ _A_ และเป็นเงื่อนไขก่อนของ _B_
และยังยืนยันว่า _p_ ต้องคงความจริงตั้งแต่เวลาของการกระทำ _A_ จนถึงเวลาของการกระทำ _B_
การละเมิดกฎนี้เรียกว่า threat และต้องแก้ทันทีด้วยการเพิ่มข้อจำกัดเชิงเวลาที่เหมาะสม
`causal_links` เก็บเป็นทูเพิล `(Action(A), precondition(p), Action(B))` ตามความหมายข้างต้น
Causal-link ยังเรียกว่า **ช่วงการป้องกัน (protection-interval)** เพราะลิงก์ _A_ --_p_--> _B_ ทำหน้าที่ปกป้อง _p_ ไม่ให้ถูกทำให้เป็นลบในช่วงเวลาระหว่าง _A_ ถึง _B_
4. **`agenda`**: เซตของเงื่อนไขก่อนที่ยังเปิดอยู่ (open-preconditions)
เงื่อนไขก่อนถือว่าเปิดอยู่หากยังไม่มีการกระทำใดในแผนทำให้มันเป็นจริง
ตัววางแผนจะพยายามลดเซตของ open preconditions ให้เป็นเซตว่าง โดยไม่ทำให้เกิดความขัดแย้ง
`agenda` เก็บเป็นทูเพิล `(precondition(p), Action(A))` โดย p เป็นเงื่อนไขก่อนของการกระทำ A

แผนที่ **สอดคล้อง (consistent)** คือแผนที่ไม่มีวัฏจักรในข้อจำกัดเชิงเวลาและไม่มีความขัดแย้งกับ causal-links
แผนที่สอดคล้องและไม่มี open preconditions คือ **คำตอบ (solution)**
<br>
<br>
ต่อไปมาดูเมธอดผู้ช่วยแบบย่อก่อนเข้าสู่อัลกอริทึมหลัก
<br>
**`expand_actions`**: สร้างการกระทำที่เป็นไปได้ทั้งหมดพร้อมการผูกตัวแปร เพื่อใช้เป็นฮิวริสติกในการเลือกเงื่อนไขก่อนที่ยังเปิดอยู่
<br>
**`find_open_precondition`**: หาเงื่อนไขก่อนจาก agenda ที่มีจำนวนการกระทำที่ทำให้เป็นจริงได้น้อยที่สุด
ฮิวริสติกนี้ช่วยสร้างข้อจำกัดเชิงเวลาที่จำเป็นและ causal-links เพื่อทำให้ปัญหาง่ายขึ้นและลดโอกาสที่จะพบ threat
<br>
**`find_action_for_precondition`**: หาแอ็กชันที่ทำให้เงื่อนไขก่อนที่กำหนดเป็นจริง พร้อมการผูกตัวแปรเท่าที่จำเป็นตามหลัก _least commitment_
หากมีได้หลายแอ็กชัน จะเลือกแอ็กชันที่มีจำนวนเอฟเฟกต์น้อยที่สุด เพื่อลดโอกาสพบ threat
<br>
**`cyclic`**: ตรวจสอบว่ากราฟมีวัฏจักรหรือไม่
<br>
**`add_const`**: เพิ่ม `constraint` ลงใน `constraints` ถ้ากราฟที่เกิดขึ้นใหม่ยังคงเป็น acyclic; มิฉะนั้นคืนค่า `constraints` เดิม
<br>
**`is_a_threat`**: ตรวจว่าค่า `effect` ที่ให้มาทำให้ `precondition` ที่ให้มาเป็นลบหรือไม่
<br>
**`protect`**: ตรวจว่าการกระทำที่ให้มาคุกคาม causal_link ที่กำหนดหรือไม่
ถ้าใช่ จะแก้ threat โดยการเลื่อนขึ้น (promotion) หรือเลื่อนลง (demotion) แล้วแต่แบบใดทำให้ข้อจำกัดเชิงเวลาไม่เป็นวงจร
ถ้าทั้ง promotion และ demotion ใช้ไม่ได้ แสดงว่าแอ็กชันที่เลือกไม่เหมาะสม หรือปัญหาวางแผนอาจแก้ไม่ได้เลย
<br>
**`convert`**: แปลงกราฟจากลิสต์ของขอบ (edges) ให้เป็นแมปแบบ `Action` : `set` เพื่อใช้ในการเรียงลำดับแบบทอพอโลยี
<br>
**`toposort`**: ฟังก์ชันเจเนอเรเตอร์ที่ให้ลำดับแบบทอพอโลยีของกราฟเป็นลิสต์ของเซต
แต่ละเซตมีหนึ่งหรือหลายแอ็กชัน
ถ้าในเซตมีมากกว่าหนึ่งแอ็กชัน แปลว่าการสลับลำดับระหว่างแอ็กชันเหล่านั้นก็ยังให้แผนที่ถูกต้อง
<br>
**`display_plan`**: แสดง `causal_links`, `constraints` และแผนลำดับบางส่วนที่ได้จาก `toposort`
<br>

เมธอด **`execute`** ทำงานตามอัลกอริทึมซึ่งสรุปได้ดังนี้:
<br>
1. เลือกเงื่อนไขก่อนที่เปิดอยู่ (เป้าหมายย่อยที่ต้องการทำให้เป็นจริง)
2. เลือกแอ็กชันที่ทำให้เงื่อนไขก่อนนั้นเป็นจริง
3. อัปเดตข้อจำกัดเชิงเวลา
4. ปกป้อง causal links ที่มีอยู่ โดยตรวจว่ามีความขัดแย้งหรือไม่
   หากมี ให้เพิ่มข้อจำกัดเชิงเวลาเพื่อแก้ threat
5. อัปเดตเซตของ open preconditions
6. กำหนดข้อจำกัดเชิงเวลาระหว่างแอ็กชันที่เลือกกับแอ็กชันถัดไป
7. เพิ่ม causal link ใหม่ระหว่างแอ็กชันที่เลือกกับเจ้าของเงื่อนไขก่อนที่เปิดอยู่
8. ตรวจสอบ causal link ใหม่ว่ามี threat หรือไม่ ถ้ามี ให้แก้ด้วย promotion หรือ demotion
   หากแก้ไม่ได้ แสดงว่าลำดับแอ็กชันปัจจุบันไม่เหมาะสม หรือปัญหาอาจแก้ไม่ได้เลย
9. ทำซ้ำขั้นตอนเหล่านี้จนกว่าเซตของ open preconditions จะว่างเปล่า

แผนแบบลำดับบางส่วนสามารถใช้สร้างแผนแบบลำดับทั้งหมดที่ถูกต้องได้หลายแบบ
ขั้นตอนนี้เรียกว่า **linearization** ของแผนลำดับบางส่วน
ตัวอย่าง linearization ทั้งหมดของแผนลำดับบางส่วนสำหรับปัญหา `socks_and_shoes` มีลักษณะดังรูป
<br>
![title](images/pop.jpg)
<br>
สามารถทำ linearization ได้หลายวิธี แต่ที่มีประสิทธิภาพคือแทนเซตของข้อจำกัดเชิงเวลาเป็นกราฟมีทิศทาง
จะเห็นได้ง่ายว่ากราฟควรไม่มีวัฏจักร เพราะถ้ามี แสดงว่าข้อจำกัดไม่สอดคล้องกัน
คุณสมบัตินี้ถูกบังคับใช้โดยเมธอด `add_const` ซึ่งจะเพิ่มข้อจำกัดใหม่ก็ต่อเมื่อยังไม่ทำให้กราฟที่มีอยู่สูญเสียความเป็น acyclic
เมธอด `protect` ก็ตรวจความเป็น acyclic ของข้อจำกัดเชิงเวลาใหม่เพื่อใช้ตัดสินใจระหว่าง promotion กับ demotion เมื่อเกิด threat
ด้วยคุณสมบัตินี้ของกราฟจากข้อจำกัดเชิงเวลาของแผนลำดับบางส่วนที่ถูกต้อง เราจึงใช้การเรียงลำดับแบบทอพอโลยีเพื่อจัดลำดับข้อจำกัดเป็นเชิงเส้นได้
การเรียงแบบทอพอโลยีอาจให้หลายคำตอบที่ถูกต้องสำหรับกราฟมีทิศทางที่ไม่มีวัฏจักรหนึ่งๆ

เมื่อเข้าใจการทำงานของ `PartialOrderPlanner` แล้ว มาลองแก้ปัญหาบางอย่างด้วยมันกัน

In [5]:
st = spare_tire()
pop = PartialOrderPlanner(st)
pop.execute()

Causal Links
(Action(PutOn(Spare, Axle)), At(Spare, Axle), Action(Finish))
(Action(Start), Tire(Spare), Action(PutOn(Spare, Axle)))
(Action(Remove(Flat, Axle)), NotAt(Flat, Axle), Action(PutOn(Spare, Axle)))
(Action(Start), At(Flat, Axle), Action(Remove(Flat, Axle)))
(Action(Remove(Spare, Trunk)), At(Spare, Ground), Action(PutOn(Spare, Axle)))
(Action(Start), At(Spare, Trunk), Action(Remove(Spare, Trunk)))
(Action(Remove(Flat, Axle)), At(Flat, Ground), Action(Finish))

Constraints
Action(Remove(Flat, Axle)) < Action(PutOn(Spare, Axle))
Action(Start) < Action(Finish)
Action(Remove(Spare, Trunk)) < Action(PutOn(Spare, Axle))
Action(Start) < Action(Remove(Spare, Trunk))
Action(Start) < Action(Remove(Flat, Axle))
Action(Remove(Flat, Axle)) < Action(Finish)
Action(PutOn(Spare, Axle)) < Action(Finish)
Action(Start) < Action(PutOn(Spare, Axle))

Partial Order Plan
[{Action(Start)}, {Action(Remove(Flat, Axle)), Action(Remove(Spare, Trunk))}, {Action(PutOn(Spare, Axle))}, {Action(Finish)}]


เราพบว่าในแผนลำดับบางส่วนที่ได้มา การกระทำ Remove(Flat, Axle) และ Remove(Spare, Trunk) อยู่ในเซตเดียวกัน
หมายความว่าลำดับการทำสองการกระทำนี้ไม่กระทบผลลัพธ์สุดท้าย
นอกจากนี้ยังเห็นว่าแอ็กชัน PutOn(Spare, Axle) ต้องทำหลังจากการกระทำ Remove ทั้งสองเสร็จสิ้น ซึ่งสมเหตุสมผลทางตรรกะ

In [6]:
sbw = simple_blocks_world()
pop = PartialOrderPlanner(sbw)
pop.execute()

Causal Links
(Action(FromTable(C, B)), On(C, B), Action(Finish))
(Action(FromTable(B, A)), On(B, A), Action(Finish))
(Action(Start), OnTable(B), Action(FromTable(B, A)))
(Action(Start), OnTable(C), Action(FromTable(C, B)))
(Action(Start), Clear(C), Action(FromTable(C, B)))
(Action(Start), Clear(A), Action(FromTable(B, A)))
(Action(ToTable(A, B)), Clear(B), Action(FromTable(C, B)))
(Action(Start), On(A, B), Action(ToTable(A, B)))
(Action(ToTable(A, B)), Clear(B), Action(FromTable(B, A)))
(Action(Start), Clear(A), Action(ToTable(A, B)))

Constraints
Action(Start) < Action(FromTable(C, B))
Action(FromTable(B, A)) < Action(FromTable(C, B))
Action(Start) < Action(FromTable(B, A))
Action(Start) < Action(ToTable(A, B))
Action(Start) < Action(Finish)
Action(FromTable(B, A)) < Action(Finish)
Action(FromTable(C, B)) < Action(Finish)
Action(ToTable(A, B)) < Action(FromTable(B, A))
Action(ToTable(A, B)) < Action(FromTable(C, B))

Partial Order Plan
[{Action(Start)}, {Action(ToTable(A, B))}, {Actio

จะเห็นว่าแผนนี้ไม่มีความยืดหยุ่นในการเลือกการกระทำ กล่าวคือ ต้องทำการกระทำตามลำดับนี้เท่านั้น จึงจะไปถึงสถานะเป้าหมายได้สำเร็จ

In [7]:
ss = socks_and_shoes()
pop = PartialOrderPlanner(ss)
pop.execute()

Causal Links
(Action(RightShoe), RightShoeOn, Action(Finish))
(Action(LeftShoe), LeftShoeOn, Action(Finish))
(Action(LeftSock), LeftSockOn, Action(LeftShoe))
(Action(RightSock), RightSockOn, Action(RightShoe))

Constraints
Action(LeftSock) < Action(LeftShoe)
Action(RightSock) < Action(RightShoe)
Action(Start) < Action(RightShoe)
Action(Start) < Action(Finish)
Action(LeftShoe) < Action(Finish)
Action(Start) < Action(RightSock)
Action(Start) < Action(LeftShoe)
Action(Start) < Action(LeftSock)
Action(RightShoe) < Action(Finish)

Partial Order Plan
[{Action(Start)}, {Action(LeftSock), Action(RightSock)}, {Action(LeftShoe), Action(RightShoe)}, {Action(Finish)}]


แผนนี้เช่นกัน ไม่มีข้อจำกัดในการเลือกถุงเท้าหรือรองเท้าก่อนหลัง
ตราบเท่าที่ทั้งสองถุงเท้าถูกสวมก่อนรองเท้าทั้งสองข้าง ทุกอย่างก็ถูกต้อง
อย่างไรก็ตาม ลองสังเกตว่ามีวิธีแก้ที่ถูกต้องแบบหนึ่ง
<br>
LeftSock -> LeftShoe -> RightSock -> RightShoe
<br>
ซึ่งอัลกอริทึมไม่สามารถค้นหาได้ เพราะมันไม่สามารถแทนด้วยแผนลำดับบางส่วนทั่วไป แต่เป็นวิธีแก้แบบลำดับทั้งหมดเฉพาะกรณี

### ความแตกต่างด้านเวลาในการรัน (Runtime differences)
ลองดูเวลาในการทำงานโดยสังเขปของอัลกอริทึมทั้งสามบนปัญหา `socks_and_shoes`

In [8]:
ss = socks_and_shoes()

In [9]:
%%timeit
GraphPlan(ss).execute()

198 µs ± 3.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
Linearize(ss).execute()

844 µs ± 23.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
PartialOrderPlanner(ss).execute(display=False)

258 µs ± 4.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


เราพบว่า `GraphPlan` เร็วกว่า `Linearize` ประมาณ 4 เท่า เพราะ `Linearize` โดยแก่นแล้วเรียกซับรูทีน `GraphPlan` อยู่เบื้องหลัง จากนั้นจึงทำทรานส์ฟอร์มบางอย่างบน planning-graph ที่แก้แล้ว
<br>
เรายังพบว่า `GraphPlan` เร็วกว่า `PartialOrderPlanner` เล็กน้อย แต่หลักๆ มาจากเมธอด `expand_actions` ใน `PartialOrderPlanner` ที่ทำให้ช้าลง เพราะมันสร้างการจัดเรียงและการผูกตัวแปรที่เป็นไปได้ทั้งหมด
<br>
หากไม่มีฮิวริสติก `PartialOrderPlanner` จะมีความเร็วอย่างน้อยเท่ากับ `GraphPlan` ถ้าไม่เร็วกว่านั้น แต่มีแนวโน้มเจอ threat และความขัดแย้งมากกว่า ซึ่งอาจใช้เวลาเพิ่มในการแก้ไข
<br>
อัลกอริทึมการวางแผนต่างๆ ทำงานได้ดีต่างกันไปตามลักษณะของปัญหา